In [1]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Testing groot to many predicates

In [2]:
predicates = {'source': ['author', 'cites'],
              'target': ['tweets', 'follows']}

In [3]:
kb = json.loads(open('src/experiments/kb.txt').readline())

In [4]:
final_results = {}
source_pred = predicates['source']
target_pred = predicates['target']

for pred in zip(source_pred, target_pred):
    final_results[f'{pred[0]}->{pred[1]}'] = ''
    final_results[f'inf_res:{pred[0]}->{pred[1]}'] = ''
    final_results[f'groot_test:{pred[0]}->{pred[1]}'] = ''
    final_results[f'tree_test:{pred[0]}->{pred[1]}'] = ''

In [5]:
kb_source = kb['yeast2']
kb_target = kb['twitter']

In [6]:
pred_target = create_pred_target(kb_target)

In [7]:
for source, target in zip(source_pred, target_pred):
    print(f'Iniciando transferência {source}->{target}')

    yeast2_dataset = datasets.load('yeast2', kb_source, target=source, seed=441773) #facts, pos, neg
    twitter_dataset = datasets.load('twitter', kb_target, target=target, seed=441773) #facts, pos, neg

    train_facts_source, train_pos_source, train_neg_source = get_train_division(yeast2_dataset)
    background_knowledge_src = boostsrl.modes(kb_source, [source], useStdLogicVariables=False, 
                                          maxTreeDepth=3, nodeSize=2, numOfClauses=8)
    model_src = boostsrl.train(background_knowledge_src, train_pos_source, train_neg_source, train_facts_source, trees=10)
    
    structured_src, new_src_struct = create_structured_trees(model_src)
    
    
    train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(twitter_dataset)
    
    start = time.time()
    final_results[f'{source}->{target}'] = genetic(new_src_struct, target, source, train_pos, 
                                             train_neg, train_facts, kb_source, kb_target, pred_target,
                                              NUM_GEN=14, pop_size=30)
    print("FIM DO GROOT: ", time.time()-start)
    
    tr_train_pos = []
    tr_train_neg = []
    tr_train_facts = []

    for i in range(0, len(train_pos)):
        tr_train_pos.extend(train_pos[i])
        tr_train_neg.extend(train_neg[i])
        tr_train_facts.extend(train_facts[i])
        
    train = [tr_train_pos, tr_train_neg, tr_train_facts]
    test = [test_pos, test_neg, test_facts]
    
    individual = get_best_individual(final_results[f'{source}->{target}'][0].population)
    refine, transfer = get_refine_transfer(individual, source, target)
    
    final_results[f'groot_test:{source}->{target}'], model = test_refine_transfer(kb_target, target, refine, transfer, train, test)
    
    final_results[f'inf_res:{source}->{target}'] = model.inference_results(target)
    
    final_results[f'tree_test:{source}->{target}'] = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
    
    save_results(final_results, source, target, 'yeast2', 'twitter')
    
    print(f"Finalizando transferência {source}->{target} com tempo {time.time()-start} s")

Iniciando transferência author->tweets


/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


GENERATION:  0
GENERATION:  1
GENERATION:  2
GENERATION:  3
GENERATION:  4
GENERATION:  5
GENERATION:  6
GENERATION:  7
GENERATION:  8
GENERATION:  9
GENERATION:  10
GENERATION:  11
GENERATION:  12
GENERATION:  13
FIM DO GROOT:  5217.052223205566
******************************************
Performing Parameter Learning
******************************************
Refine
0;;author(A, B) :- fauthor(C, B), dmhead(C, D).;true;true
0;true;fauthor(A, B).;false;false
0;false;fauthor(A, E), lauthor(F, E).;true;true
0;false,true;lauthor(A, E), dmhead(F, G).;false;true
0;false,true,false;chem(A, H), chem(F, H).;false;false
0;false,false;qmhead(A, I), chem(A, J).;true;true
0;false,false,true;qmhead(K, I), dmhead(K, L).;false;false
0;false,false,false;qmhead(A, M).;false;false
1;;author(A, B) :- fauthor(A, C), lauthor(D, C).;false;true
1;false;dmhead(A, E), cites(A, F).;true;true
1;false,true;qmhead(A, G), chem(A, H).;true;true
1;false,true,true;dmhead(I, E), cites(J, I).;false;false
1;false,true,fal

Results
   AUC ROC   = 0.505197
   AUC PR    = 0.510393
   CLL	      = -0.694731
   Precision = nan at threshold = 0.5
   Recall    = 0.0
   F1        = nan


Total learning time: 38.416 seconds
Total inference time: 5.982 seconds
AUC ROC: 0.505197


Results scoring model
   AUC ROC   = 0.501487
   AUC PR    = 0.507587
   CLL	      = -0.694969
   Precision = nan at threshold = 0.5
   Recall    = 0.0
   F1        = nan


Total scoring time: 5.248 seconds
Parameter learned model CLL: -0.694969


Structure after Parameter Learning
[['tweets(A, B)', {'': 'follows(C, D)'}, {'true': [0.5, 682, 698], 'false': [0.0, 0, 0]}], ['tweets(A, B)', {'': 'follows(A, C)', 'true': 'accounttype(A, D)', 'true,true': 'follows(E, C)', 'false': 'accounttype(A, F)', 'false,true': 'accounttype(G, H)'}, {'true,true,true': [17.406, 600, 612], 'true,true,false': [0.0, 0, 0], 'true,false': [0.0, 0, 0], 'false,true,true': [6.479, 82, 86], 'false,true,false': [0.0, 0, 0], 'false,false': [0.0, 0, 0]}], ['tweets(A, B)

Results
   AUC ROC   = 0.570212
   AUC PR    = 0.539378
   CLL	      = -0.686413
   Precision = nan at threshold = 0.5
   Recall    = 0.0
   F1        = nan


Total learning time: 30.514 seconds
Total inference time: 4.654 seconds
AUC ROC: 0.570212


Results scoring model
   AUC ROC   = 0.561052
   AUC PR    = 0.533619
   CLL	      = -0.687471
   Precision = nan at threshold = 0.5
   Recall    = 0.0
   F1        = nan


Total scoring time: 4.257 seconds
Parameter learned model CLL: -0.687471


Structure after Parameter Learning
[['follows(A, B)', {'': 'tweets(A, C)', 'true': 'accounttype(A, D)', 'true,true': 'accounttype(B, E)'}, {'true,true,true': [17.593, 619, 619], 'true,true,false': [0.0, 0, 0], 'true,false': [0.0, 0, 0], 'false': [6.671, 89, 89]}], ['follows(A, B)', {'': 'tweets(C, D)'}, {'true': [18.815, 708, 708], 'false': [0.0, 0, 0]}], ['follows(A, B)', {'': 'accounttype(C, D)'}, {'true': [18.815, 708, 708], 'false': [0.0, 0, 0]}], ['follows(A, B)', {'': 'tweets(A, C)', 'true'

In [22]:
save_results(final_results, source, target, 'yeast2', 'twitter')